<a href="https://colab.research.google.com/github/PriyanshuNaredi/RDP/blob/main/WineHQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created with the help of https://github.com/PradyumnaKrishna

In [ ]:
#@title **Create User**
#@markdown Enter Username and Password

import os

username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}

print("Creating User and Setting it up")

# Creation of user
os.system(f"useradd -m {username}")

# Add user to sudo group
os.system(f"adduser {username} sudo")
    
# Set password of user to 'root'
os.system(f"echo '{username}:{password}' | sudo chpasswd")

# Change default shell from sh to bash
os.system("sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd")

print(f"User created and configured having username `{username}` and password `{password}`")

In [ ]:
#@title **RDP**
#@markdown  It takes 4-5 minutes for installation

import os
import subprocess

#@markdown  Visit http://remotedesktop.google.com/headless and copy the command after Authentication

CRP = "" #@param {type:"string"}

#@markdown Enter a Pin (more or equal to 6 digits)
Pin = 123456 #@param {type: "integer"}

#@markdown Autostart Notebook in RDP
Autostart = False #@param {type: "boolean"}


class CRD:
    def __init__(self, user):
        os.system("apt update")
        self.installCRD()
        self.installDesktopEnvironment()
        self.installGoogleChorme()
        self.finish(user)
        print("\nRDP created succesfully move to https://remotedesktop.google.com/access")

    @staticmethod
    def installCRD():
        print("Installing Chrome Remote Desktop")
        subprocess.run(['wget', 'https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['dpkg', '--install', 'chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def installDesktopEnvironment():
        print("Installing Desktop Environment")
        os.system("export DEBIAN_FRONTEND=noninteractive")
        os.system("apt install --assume-yes xfce4 desktop-base xfce4-terminal")
        os.system("bash -c 'echo \"exec /etc/X11/Xsession /usr/bin/xfce4-session\" > /etc/chrome-remote-desktop-session'")
        os.system("apt remove --assume-yes gnome-terminal")
        os.system("apt install --assume-yes xscreensaver")
        os.system("systemctl disable lightdm.service")

    @staticmethod
    def installGoogleChorme():
        print("Installing Google Chrome")
        subprocess.run(["wget", "https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(["dpkg", "--install", "google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def finish(user):
        print("Finalizing")
        if Autostart:
            os.makedirs(f"/home/{user}/.config/autostart", exist_ok=True)
            link = "https://colab.research.google.com/github/PradyumnaKrishna/Colab-Hacks/blob/master/Colab%20RDP/Colab%20RDP.ipynb"
            colab_autostart = """[Desktop Entry]
Type=Application
Name=Colab
Exec=sh -c "sensible-browser {}"
Icon=
Comment=Open a predefined notebook at session signin.
X-GNOME-Autostart-enabled=true""".format(link)
            with open(f"/home/{user}/.config/autostart/colab.desktop", "w") as f:
                f.write(colab_autostart)
            os.system(f"chmod +x /home/{user}/.config/autostart/colab.desktop")
            os.system(f"chown {user}:{user} /home/{user}/.config")

        os.system(f"adduser {user} chrome-remote-desktop")
        command = f"{CRP} --pin={Pin}"
        os.system(f"su - {user} -c '{command}'")
        os.system("service chrome-remote-desktop start")
        

        print("Finished Succesfully")


try:
    if CRP == "":
        print("Please enter authcode from the given link")
    elif len(str(Pin)) < 6:
        print("Enter a pin more or equal to 6 digits")
    else:
        CRD(username)
except NameError as e:
    print("'username' variable not found, Create a user first")

In [ ]:
#@title Methode 1 (recommended)
#@title **Google Drive Mount**
#@markdown Google Drive used as Persistance HDD for files.<br>
#@markdown Mounted at `user` Home directory inside drive folder

#@markdown Choose a method (GDFuse Recommended)
mount_method = "GDFuse" #@param ["GDFuse", "Native"]

try:
    mountpoint = f"/home/{username}/drive"
except NameError:
    print("'username' variable not found, mounting at `/content/drive`")
    mountpoint = '/content/drive'

def mount_gdfuse():
    ! apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    ! add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
    ! apt-get update -qq 2>&1 > /dev/null
    ! apt-get -y install -qq google-drive-ocamlfuse fuse
    import getpass
    from google.colab import auth
    from oauth2client.client import GoogleCredentials


    auth.authenticate_user()
    creds = GoogleCredentials.get_application_default()
    ! google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
    vcode = getpass.getpass()
    ! echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

    ! mkdir -p $mountpoint
    ! google-drive-ocamlfuse -o allow_other $mountpoint

def mount_native():
    from google.colab import drive
    drive.mount(mountpoint)

if mount_method == "Native":
    mount_native()
if mount_method == "GDFuse":
    mount_gdfuse()


In [ ]:
#@title Google Drive
#@markdown Recquies to be authenticated twice
! apt-get install -y -qq software-properties-common python-software-properties module-init-tools
! add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
! apt-get update -qq 2>&1 > /dev/null
! apt-get -y install -qq google-drive-ocamlfuse fuse

import getpass
from google.colab import auth
from oauth2client.client import GoogleCredentials

try:
    mountpoint = f"/home/{username}/drive"
except NameError:
    print("'username' variable not found, mounting at `/content/drive`")
    mountpoint = '/content/drive'

auth.authenticate_user()
creds = GoogleCredentials.get_application_default()
! google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
! echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

! mkdir -p $mountpoint
! google-drive-ocamlfuse -o allow_other $mountpoint

In [ ]:
#@title WineHQ
#@markdown For Running Windows Application On your RDP

!sudo apt update
!sudo dpkg --add-architecture i386
!wget -qO- https://dl.winehq.org/wine-builds/Release.key | sudo apt-key add -
!sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv F987672F
!sudo apt-add-repository 'deb https://dl.winehq.org/wine-builds/ubuntu/ bionic main'
!sudo apt-get update
!sudo apt-get install --install-recommends winehq-stable

In [ ]:
#@title **Anti-Shutdown**
#@markdown Run this when you're playing Minecraft to prevent the RDP from shutting down. When you need to restore or backup MC saves, stop this block, restore/backup, and start this block again. Note that RDP will always shut down after 12 hours, even if still being used.<br>

while True:pass

In [ ]:
#@title **Colab Shutdown**

#@markdown To Kill NGROK Tunnel
NGROK = False #@param {type:'boolean'}

#@markdown To Unmount GDrive
GDrive = False #@param {type:'boolean'}

#@markdown To Sleep Colab
Sleep = True #@param {type:'boolean'}

if NGROK:
    ! killall ngrok

if GDrive:
    with open('/content/unmount.py', 'w') as unmount:
        unmount.write("""from google.colab import drive
drive.flush_and_unmount()""")
    
    try:
        if user:
            ! runuser $user -c 'python3 /content/unmount.py'
    except NameError:
        print("Google Drive not Mounted")

if Sleep:
    from time import sleep
    sleep(43200)